<center>
<h1>Mustererkennung und Machine Learning</h1>

<h3> Wintersemester 2017/2018, 10th Exercise Sheet</h3>
<h4>Konstantin Jaehne, Luis Herrmann; Dozent: Raúl Rojas</h4>

<hr style='height:1px'>
</center>

# Exercise 1

As usual, we read in the digit dataset fron the respective files:

In [333]:
%matplotlib inline
import sys
import numpy as np
import numpy.random as nprd
import random as rd
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML

folderpath = '../'
filename = 'spambase.data'
data = pd.read_csv(folderpath+filename, header=None).as_matrix()

spam = []
nospam = []
for sample in data:
    if(sample[57] == 1):
        spam.append(sample[:57])
    elif(sample[57] == 0):
        nospam.append(sample[:57])
print('Data contains ' + str(len(spam)) + ' data points of spam and ' + str(len(nospam)) + ' points of non-spam.')
rd_nospam = nprd.permutation(nospam)
rd_spam = nprd.permutation(spam)
train = (rd_nospam[:int(0.8 * len(nospam))], rd_spam[:int(0.8 * len(spam))])
test = (rd_nospam[int(0.8 * len(nospam)):], rd_spam[int(0.8 * len(spam)):])

Data contains 1813 data points of spam and 2788 points of non-spam.


First of all, we implement a AdaBoost Binary Classifier:

In [946]:
class AdaBoostBinary:
    def __init__(self, data, classifiers, labels=None, itlimit=None):
        """
        data:
            Type: List/Tuple
            A list/tuple (l1,l2), where l1 and l2 are lists containing vectors assigned to class 1 and 2, respectively.
        classifiers:
            Type: List/Tuple
            A list/tuple k1,...,kn wheren ki is a binary classifer which for a matrix X[i,j] in which
            X[i,:] is a feature vector, returns a vector for which v[i] is -1 or 1.
        labels (optional):
            Type: List/Tuple
            A list/tuple (i1,i2) of two printable identifiers associated with each class
        itlimit (optional):
            Type: Integer
            Maximum number of iterations, must be smaller than length of classifier list.
        """
        if(not(labels is None)):
            if(len(labels) == 2):
                self.labels = np.array(labels)
            else:
                raise Exception('You must provide exactly two labels!')
        else:
            self.labels = np.array([0, 1], dtype=int)
        if(not(data is None) and not(classifiers is None)):
            self.train(data, classifiers, itlimit)
            
    def train(self, data, classifiers, itlimit=None):
        if(not(itlimit is None) and itlimit > len(classifiers)):
            raise Exception('itlimit has to be smaller than the number of classifiers!')
        """The k-classifiers have to return a -1, or 1"""
        k = classifiers
        Plen, Nlen = len(data[0]), len(data[1])
        x = np.vstack(data)
        x = np.hstack([x, np.repeat(1, x.shape[0])[:,np.newaxis]])
        if(itlimit is None):
            itlimit = len(classifiers)
        #Create label vector:
        y = np.repeat(1, x.shape[0])
        y[:x.shape[0]] = 1
        y[x.shape[0]:] = -1
        #Create classifier assignment matrix with R[i,j] <- k_j(x_i) == y_i
        R = np.zeros([x.shape[0], len(k)])
        for i in range(len(k)):
            R[:,i] = (k[i].classify(x) == y)
        #Initialize weight vector
        w = np.repeat(1, x.shape[0])
        self.alpha = []
        self.C = []
        for i in range(itlimit):
            #Select classifier that minimizes E_neq
            indexmin= np.argmin([np.sum(w[np.argwhere(R[:,i] == 0)]) for i in range(len(k))])
            E_eq = np.sum(w[np.argwhere(R[:,indexmin] == 1)], axis=0)
            E_neq = np.sum(w[np.argwhere(R[:,indexmin] == 0)], axis=0)
            #Recalculate weight vector
            w = np.exp(-np.multiply(y, self.classify(x)))
            self.alpha.append(1/2 * np.log(E_eq / E_neq))
            self.C.append(k[indexmin])
            #Remove selected classifier from list:
            k.pop(indexmin)
            np.delete(R, indexmin, axis=1)
            sys.stdout.write(str(i+1) + ' iterations\r')
            
    def classify(self, data):
        s = np.zeros(data.shape[0])
        for i in range(len(self.C)):
            s += np.multiply(self.alpha[i], self.C[i].classify(data))
        cls = np.zeros(s.shape, self.labels.dtype)
        cls[s < 0.5] = self.labels[0]
        cls[s >= 0.5] = self.labels[1]
        return(np.array(s > 0.5, dtype=int))
        

We also import the LogisticClassifier-class from a past exercise sheet and apply some minor modifications to allow for outputs of labels -1 and +1.

In [732]:
class LogisticClassifier:
    def __init__(self, data=None, labels=None, itlimit=1000, tol=1e-19, coords=None):
        """
        data:
            Type: List/Tuple
            A list/tuple (l1,l2), where l1 and l2 are lists containing vectors assigned to class 1 and 2, respectively.
        labels (optinal):
            Type: List/Tuple
            A list/tuple (i1,i2) of two printable identifiers associated with each class
        itlimit (optional):
            Type: Integer
            Maximum number of iterations, before training is forcefully terminated.
        tol (optional)
            Type: Float
            Minimum rate of improvement that is acceptable. If improvement drops below tolerance, stop descent.
        """
        if(not(labels is None)):
            if(len(labels) == 2):
                self.labels = np.array(labels)
            else:
                raise Exception('You must provide exactly two labels!')
        else:
            self.labels = np.array([0, 1], dtype=int)
        if(not(data is None)):
            if(len(data) != 2):
                raise Exception('Data does not follow specified format!')
            if(len(data[0]) == 0 or len(data[1]) == 0):
                raise Exception('Data lists must contain at least one sample!')
            self.train(data, itlimit, tol, coords)
    
    def train(self, data, itlimit=1000, tol=1e-19, coords=None):
        Plen, Nlen = len(data[0]), len(data[1])
        self.x = np.vstack(data)
        #Extract only desired coordinates
        if(not(coords is None)):
            self.coords = np.array(coords, dtype=int)
            self.x = self.x[:,self.coords]
        else:
            self.coords = coords
        #Calculate the variance for all features and set to 1 if zero. This will lead to no normalization.
        self.variance = np.var(self.x, axis=0)[np.newaxis,:]
        self.variance[np.where(self.variance == 0)] = 1
        #Calculate the mean for all features
        self.mean = np.mean(self.x, axis=0)[np.newaxis,:]
        #Normalize data
        self.x = self.normalize(self.x)
        #Add column of 1s
        self.x = np.hstack([np.ones(Plen+Nlen)[:,np.newaxis], self.x])
        self.y = np.append(np.repeat(1.0, Plen), np.repeat(-1.0, Nlen))
        #self.y = np.append(np.repeat(1.0, Plen), np.repeat(0.0, Nlen))
        #self.w = self.x[rd.randint(0, self.x.shape[1]-1)]
        self.w = nprd.random(self.x.shape[1])
        self.offset = 0
        stepsize = 1e+1
        t = 0
        while(t < itlimit):
            weights = self.y * self.sigmoid(np.multiply(-self.y, np.dot(self.x, self.w)))
            #weights = self.y - self.sigmoid(np.dot(self.x, self.w))
            gradient =  np.sum(np.multiply(self.x, weights[:,np.newaxis]), axis=0)
            #Back-tracking: The measure for improvements is the avg of probabilitis rather than the likelihood function
            while(self.avgprob(self.w + stepsize * gradient) < self.avgprob(self.w + (stepsize / 2) * gradient)):
                stepsize /= 2
            #If the improvement drops below a certain tolerance, quit ascent
            if(self.avgprob(self.w + stepsize * gradient) - self.avgprob(self.w) < tol):
                break
            self.w += stepsize * gradient
            t += 1
            sys.stdout.write(str(t) + ' iterations, stepsize %.3e' % stepsize + '\r')
        sys.stdout.write('\nTraining complete!\n')
    
    def normalize(self, data):
        """Normalizes the data to allow for faster convergence of the gradient ascent"""
        return((data - self.mean) / self.variance)

    def classify(self, data):
        """Classifies a vector"""
        prob = self.prob(data)
        cls = np.zeros(prob.shape, self.labels.dtype)
        cls[prob > 0.5] = self.labels[0]
        cls[prob <= 0.5] = self.labels[1]
        return(cls)
    
    def prob(self, data):
        """Returns probabilities for a vector to be in the positive class"""
        if(not(self.coords is None)):
            data = data[:,self.coords]
        return(self.sigmoid(np.dot(np.hstack([self.normalize(data), np.ones([data.shape[0],1])]), self.w) - self.offset))
    
    def avgprob(self, w):
        """Given a perceptron w, returns the average probability for the data to be in the positive class"""
        return(np.sum(self.sigmoid(np.multiply(self.y, np.dot(self.x, w))))/self.x.shape[0])
    
    def likelihood(self, w):
        """Given a perceptron w, returns the likelihood for the data to be in the positive class"""
        return(np.prod(self.sigmoid(np.multiply(self.y, np.dot(self.x, w)))))
    
    def sigmoid(self, x):
        """The sigmoidal functinon as defined by the numerically more stable tanh"""
        return(1/2 * (1 + np.tanh(1/2 * x)))

In [904]:
from IPython.display import display, HTML

def runtest(C, data, silent=False):
    n = len(C.labels)
    confusionMat = np.zeros([n, n], dtype=int)
    for i in range(n):
        result = C.classify(data[i])
        for j in range(len(result)):
            confusionMat[i,C.labels == result[j]] += 1
    err = 1-sum(np.diag(confusionMat))/np.sum(confusionMat)
    if(not(silent)):
        print('-The confusion matrix is given by:')
        html = pd.DataFrame(confusionMat,index=C.labels, columns=C.labels).to_html()
        display(HTML(html))
        print('-The error rate is: ' + str(err) + '\n')
    return(confusionMat, err)

We then proceed to create a list of weak classifiers, for which the error rate is smaller than 50%:

In [908]:
C = []
ranges = [list(range(5*i, 5*(i+1))) for i in range(10)] + [list(range(50,57))]
for i in range(len(ranges)):
    print('Classifier ' + str(i) + ':\n')
    while(True):
        Ci = LogisticClassifier(train, labels=(-1,1), itlimit=1000, coords= ranges[i])
        confusionMat, err  = runtest(Ci, test, silent=True);
        if(err < 0.5):
            print('-The error rate is: ' + str(err))
            break
    C.append(Ci)

Classifier 0:

5 iterations, stepsize 1.562e-01
Training complete!
6 iterations, stepsize 9.766e-03
Training complete!
3 iterations, stepsize 1.953e-02
Training complete!
3 iterations, stepsize 1.250e+00
Training complete!
10 iterations, stepsize 1.192e-06
Training complete!
3 iterations, stepsize 3.906e-02
Training complete!
3 iterations, stepsize 2.500e+00
Training complete!
4 iterations, stepsize 3.906e-02
Training complete!
4 iterations, stepsize 3.125e-01
Training complete!
6 iterations, stepsize 1.526e-04
Training complete!
5 iterations, stepsize 1.526e-04
Training complete!
5 iterations, stepsize 2.441e-03
Training complete!
6 iterations, stepsize 1.953e-02
Training complete!
4 iterations, stepsize 6.250e-01
Training complete!
3 iterations, stepsize 2.500e+00
Training complete!
5 iterations, stepsize 3.125e-01
Training complete!
4 iterations, stepsize 7.812e-02
Training complete!
4 iterations, stepsize 1.562e-01
Training complete!
12 iterations, stepsize 3.052e-04
Training compl

We can feed these to our implementation of Adaboost to obtain a new classifier:

In [947]:
CBoost = AdaBoostBinary(train, C.copy())
runtest(CBoost, test)

-The confusion matrix is given by:


,0,1
0,382,176
1,64,299


-The error rate is: 0.260586319218



(array([[382, 176],
        [ 64, 299]]), 0.26058631921824105)